#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [1]:
# 标准库
import os
import sys

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
from src.annotator.sem_tagger import SEMTagger

#### **2. 加载模型**

加载大模型语义标注 API 接口和 PyMUSAS

In [2]:
# === 加载模型：LLM USAS 赋码集 ===

# 语种：汉语
lang = 'chinese'

# 赋码集：USAS
# https://ucrel.lancs.ac.uk/usas/
# https://ucrel.lancs.ac.uk/usas/usas_guide.pdf
tagset = 'usas'

# 标注模式：LLM API
mode = 'llm'

# 大模型：kimi-k2.5 | glm-5 | deepseek-v3.2 | qwen3-max
llm_models = ['kimi-k2.5','glm-5']

# 登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账号
# 并在 llm_corpus_annotation/config 文件中设置 LLM_API_KEY=sk-********

llm_tagger = {}
for model in llm_models:
    llm_tagger[model] = SEMTagger(
        lang=lang,
        tagset=tagset,
        mode=mode,
        llm_model=model,
    )
print('LLM 中文语义标注标注模型加载完毕！')

LLM 中文语义标注标注模型加载完毕！


In [3]:
# === 加载模型：PyMUSAS USAS 赋码集 ===

# 标注模式：PyMUSAS 本地模型
mode = 'local'

# PyMUSAS 模型：Rule-based Tagger
# https://ucrel.github.io/pymusas/usage/how_to/tag_text_with/rule_based_tagger
# https://github.com/UCREL/Multilingual-USAS/tree/master/Chinese
# 首次加载需从网络将模型下载至本地

pymusas_tagger = SEMTagger(
    lang=lang,
    tagset=tagset,
    mode=mode,
)
print('PyMUSAS USAS 中文语义标注标注模型加载完毕！')

PyMUSAS USAS 中文语义标注标注模型加载完毕！


#### **3. 语义标注**

使用大模型和 PyMUSAS 标注汉语语义

In [4]:
# === 测试数据 ===

# 选自《鹿鼎记》

zh_text = """
总舵主道：“这里只有你我二人，不用怕难为情。
他本来想说：“这小姑娘在宫里耽得久了，太过危险，倘若给人发觉，那可糟糕之极。”
小兄弟的武功和茅爷不是一路，不知尊师是哪一位？”
"""

In [5]:
# === 调用大模型 API 和 PyMUSAS：开始语义标注 ===

# --- 注意 ---
# 为节省 API 调用成本
# 大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 若需测试 API 连接是否正常
# 可更换测试数据，重新标注

print(f'=== 中文语义标注 ===\n')

# 逐行遍历所有数据
for index, text in enumerate(zh_text.strip().splitlines()):
    print(f'[ID]: {index+1:05d}')
    print(text)
    print(f'{"-" * 60}')

    # 大模型语义标注
    for model in llm_models:
        print(f'[{model}]:')
        llm_anno = llm_tagger[model].tag(text.strip())
        llm_tags = [(tok, usas) for (tok, usas) in zip(llm_anno['tok'], llm_anno['usas'])]
        print(llm_tags)
        print(f'{"-" * 60}')

    # PyMUSAS 语义标注
    print(f'[musas]:')
    musas_anno = pymusas_tagger.tag(text.strip())
    musas_tags = [(tok, usas) for (tok, usas) in zip(musas_anno['tok'], musas_anno['usas'])]
    print(musas_tags)
    print(f'{"-" * 60}')

=== 中文语义标注 ===

[ID]: 00001
总舵主道：“这里只有你我二人，不用怕难为情。
------------------------------------------------------------
[kimi-k2.5]:
[('总舵主', 'S7.1+/S2'), ('道', 'Q2.2'), ('：', 'PUNCT'), ('“', 'PUNCT'), ('这里', 'M6'), ('只有', 'A14'), ('你我', 'Z8/S1.1.2+'), ('二人', 'N1/S2'), ('，', 'PUNCT'), ('不用', 'S6-'), ('怕', 'E5-'), ('难为情', 'E4.2-/S1.2.4-'), ('。', 'PUNCT'), ('”', 'PUNCT')]
------------------------------------------------------------
[glm-5]:
[('总舵主', 'G1.1/S2/S7.1+'), ('道', 'Q2.2'), ('：', 'PUNCT'), ('“', 'PUNCT'), ('这里', 'M7'), ('只有', 'N5.1-/Z5'), ('你', 'Z8'), ('我', 'Z8'), ('二人', 'N1/S5'), ('，', 'PUNCT'), ('不用', 'S6-'), ('怕', 'E5-'), ('难为情', 'E4.1-/S1.2.4-'), ('。', 'PUNCT')]
------------------------------------------------------------
[musas]:
[('总', 'A14'), ('舵', 'M4'), ('主', 'A11.1+'), ('道', 'M3'), ('：', 'PUNCT'), ('“', 'PUNCT'), ('这里', 'Z8/M6'), ('只有', 'Z5'), ('你', 'Z8'), ('我', 'Z8'), ('二', 'N1'), ('人', 'S2'), ('，', 'PUNCT'), ('不', 'Z6'), ('用', 'A1.5.1'), ('怕', 'E5-'), ('难为情', 'Z99'), ('。', 'P